In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import Config
import warnings
import matplotlib.pyplot as plt
import datetime
import Directional_Strategy_Manager
from os import listdir
from os.path import isfile, join

warnings.filterwarnings("ignore")
client=MongoClient(Config.DB_Hostname,Config.DB_Port)

class PnL_Estimator:
    
    def __init__(self, start_date, end_date, prediction_date, underlying):
        
        self.start_date=start_date #2017-01-01
        self.end_date=end_date #2017-01-01
        self.prediction_date=prediction_date #2017-01-01
        self.underlying=underlying #NIFTY,BANKNIFTY
        
        if self.start_date>=self.prediction_date:
            raise Exception("Start date has to be before Prediction Date")
        
        self.underlying_prices=pd.DataFrame(client[f'{Config.Data_DB}'][f'{self.underlying}OHLC'].find())
        self.underlying_prices=self.underlying_prices.sort_values(['date', 'batch_id']).drop(columns=['_id'])
        print("Prices Downloaded")
                
        self.days_to_expiry=pd.DataFrame(client[f'{Config.Data_DB}']['Days_To_Expiry'].find({"underlying":self.underlying}))
        self.days_to_expiry=self.days_to_expiry.sort_values(['date']).drop(columns=['_id'])
        print("Days To Expiry Downloaded")
        
        self.underlying_simulated_parameters=pd.DataFrame(client[f'{Config.Data_DB}'][f'{self.underlying}_Simulated_Parameters'].find())
        self.underlying_simulated_parameters=self.underlying_simulated_parameters.sort_values(['date', 'strategy_variant']).drop(columns=['_id'])
        self.underlying_simulated_parameters=self.underlying_simulated_parameters[['date', 'strategy_variant', 'pos_param', 'mov_param', 'prof_param', 'sim_vol']]
        print("Simulated Parameters Downloaded")        
        
        self.merged_parameters=pd.DataFrame(index=self.underlying_simulated_parameters.date.unique())
        self.underlying_simulated_parameters.set_index('date',inplace=True)
        self.days_to_expiry.drop(columns=['underlying'],inplace=True)
        self.days_to_expiry.set_index('date',inplace=True)
        self.underlying_prices.set_index('date',inplace=True)
        self.sim_vols=pd.merge(self.underlying_simulated_parameters[self.underlying_simulated_parameters.strategy_variant==1],
                              self.underlying_simulated_parameters[self.underlying_simulated_parameters.strategy_variant==13],
                              left_index=True,right_index=True)
        self.sim_vols=self.sim_vols[['sim_vol_x','sim_vol_y']]
        self.sim_vols.columns=['current_vol','next_vol']

        for i in range(1,self.underlying_simulated_parameters.strategy_variant.max()+1):
            temp=self.underlying_simulated_parameters[self.underlying_simulated_parameters.strategy_variant==i].drop(columns=['strategy_variant','sim_vol'])
            cols=temp.columns
            temp.columns=[f'{i}_' + col for col in cols]
            self.merged_parameters=pd.merge(self.merged_parameters,temp,left_index=True,right_index=True)
        
        self.merged_parameters=self.merged_parameters[(self.merged_parameters.index>=self.start_date)&
                                                     (self.merged_parameters.index<=self.end_date)]
        self.days_to_expiry=self.days_to_expiry[(self.days_to_expiry.index>=self.start_date)&
                                                     (self.days_to_expiry.index<=self.end_date)]
        self.underlying_prices=self.underlying_prices[(self.underlying_prices.index>=self.start_date)&
                                                     (self.underlying_prices.index<=self.end_date)]
        self.sim_vols=self.sim_vols[(self.sim_vols.index>=self.start_date)&(self.sim_vols.index<=self.end_date)]
        
        self.merged_parameters.sort_index(inplace=True)
        self.days_to_expiry.sort_index(inplace=True)
        self.underlying_prices.sort_index(inplace=True)
        self.sim_vols.sort_index(inplace=True)
        
        if self.end_date not in self.merged_parameters.index:
            self.merged_parameters.loc[self.end_date]=[np.nan]*len(self.merged_parameters.columns)
        
        datelist1=self.merged_parameters.index.unique()
        datelist2=self.underlying_prices.index.unique()
        datelist3=self.sim_vols.index.unique()
        datelist4=self.days_to_expiry.index.unique()

        for d in datelist1:
            if d!=self.end_date and (d not in datelist2 or d not in datelist2 or d not in datelist3):
                raise Exception("Dates do not match")
                
        self.previous_close_map={}
        for d1 in datelist1:
            temp=None
            for d2 in datelist2:
                if d2==d1:
                    break
                temp=d2

            self.previous_close_map[d1]=temp

        df_temp=pd.merge(self.merged_parameters,self.days_to_expiry,left_index=True,right_index=True)
        temp_cols=self.merged_parameters.columns

        df_temp=df_temp.shift().dropna()

        for col in temp_cols:

            temp=int(col[:col.find('_')])
            temp_str=col[col.find('_'):]

            if (temp-1)%24<12:
                df_temp[col]=np.where(df_temp.current_week==1,df_temp[str(temp+12)+temp_str],df_temp[col])

        self.merged_parameters=df_temp.drop(columns=['current_week','next_week'])
        del self.underlying_simulated_parameters
        print("PnL Estimator Initialized")
    
    def Predict(self):
        
        strat_variants=pd.DataFrame(client[f'{Config.Data_DB}']['Strategy_Variants'].find())
        strat_variants=strat_variants.drop(columns=['_id','underlying']).set_index('strategy_variant')
        num_strategies=strat_variants.index.max()
        flag=False
        for date in self.merged_parameters.index:

            if date==self.prediction_date:
                flag=True

            if not flag:
                continue

            summary={'date':[], 'strategy_variant':[], 'pos_param':[], 'mov_param':[], 'prof_param':[], 
                     'PnL':[], 'Drawdown_Count':[], 'Trade_Count':[], 'first_drawdown_pnl':[], 'second_drawdown_pnl':[]}

            snap=self.merged_parameters.loc[date]
            snap=np.array(snap).reshape(num_strategies,3)

            prices=self.underlying_prices[self.underlying_prices.index==date].sort_values('batch_id')
            prices.batch_id=prices.batch_id.astype(int)

            if len(prices.dropna())!=375 :
                if date!= self.end_date:
                    raise Exception(f"Prices incomplete for {date}")

            try:
                vols=self.sim_vols.loc[date]
                if len(vols.dropna())!=2:
                    raise Exception(f"Sim vol not available for {date}")
                else:
                    vols=list(vols)
            except:
                if date!= self.end_date:
                    raise Exception(f"Sim vol not available for {date}")
                else:
                    vols=[0.2,0.2]

            try:
                expiry=self.days_to_expiry.loc[date]
                if len(expiry.dropna())!=2:
                    raise Exception(f"Days to expiry not available for {date}")
                expiry=list(expiry)
            except:
                raise Exception(f"Days to expiry not available for {date}")

            try:
                previous_close=self.underlying_prices[self.underlying_prices.index==self.previous_close_map[date]].sort_values('batch_id').dropna().close[-1]
            except Exception as e:
                raise Exception(f"Could not initialize previous close {e}")

            for variant, params in enumerate(snap):

                strategy_params={}
                blueprint=strat_variants.loc[variant+1].to_dict()

                strategy_params["strategy_type"]=blueprint["strategy_type_parameter"]
                strategy_params["strategy_version"]=blueprint["strategy_version_parameter"]
                strategy_params["previous_close"]=previous_close
                strategy_params["initial_time_parameter"]=10
                strategy_params["position_ratio"]=blueprint["position_ratio"]

                if blueprint["week_parameter"]==1:
                    strategy_params["current_week"]=True
                    strategy_params["sim_vol"]=vols[0]
                    strategy_params["days_to_expiry"]=expiry[0]

                else:
                    strategy_params["current_week"]=False
                    strategy_params["sim_vol"]=vols[1]
                    strategy_params["days_to_expiry"]=expiry[1]

                strategy_params["position_parameter"]=params[0]*blueprint['position_multiplier']
                strategy_params["abstinence_parameter"]=params[0]*blueprint["abstinence_multiplier"]
                strategy_params["straight_abstinence_parameter"]=params[0]*blueprint["straight_abstinence_multiplier"]
                strategy_params["straight_reversal_parameter"]=params[0]*blueprint["straight_reversal_multiplier"]

                strategy_params["directional_move_parameter"]=params[1]*blueprint['directional_move_multiplier']
                strategy_params["extreme_move_parameter"]=params[1]*blueprint["extreme_move_multiplier"]
                strategy_params["initial_move_parameter"]=params[1]*blueprint["initial_move_multiplier"]

                strategy_params["take_profit_parameter"]=params[2]*blueprint['take_profit_multiplier']
                strategy_params["hedge_manage_parameter"]=params[2]*blueprint['hedge_manage_multiplier']
                strategy_params["stop_loss_parameter"]=params[2]*blueprint['stop_loss_multiplier']

                res=Directional_Strategy_Manager.Test(strategy_params,prices,date)

                summary['date'].append(date)
                summary['strategy_variant'].append(variant+1)
                summary['pos_param'].append(params[0])
                summary['mov_param'].append(params[1])
                summary['prof_param'].append(params[2])
                summary['PnL'].append(res['PnL'])
                summary['Drawdown_Count'].append(res['Drawdown_Count'])
                summary['Trade_Count'].append(res['Trade_Count'])
                summary['first_drawdown_pnl'].append(res['first_drawdown_pnl'])
                summary['second_drawdown_pnl'].append(res['second_drawdown_pnl'])

            print(client[f'{Config.Data_DB}'][f'{self.underlying}_Predicted_Parameters'].delete_many({"date":date}).deleted_count,f" documents deleted for {date}")
            print(len(client[f'{Config.Data_DB}'][f'{self.underlying}_Predicted_Parameters'].insert_many(pd.DataFrame(summary).to_dict('records')).inserted_ids),f" documents entered for {date}\n")

In [2]:
start_date = '2015-01-12' #starting data point best left untouchedd
end_date = '2024-01-19' # Last day for which you want prediction - should be next trading day
prediction_date = '2024-01-18' # should be after startdate and before end date. Only dates >= are processed and pushed into DB
#side note if you should also include todays's date for the daily run as pnl would have not been calculated yesterday, obviously
underlying = 'NIFTY' #NIFTY/BANKNIFTY

P=PnL_Estimator(start_date, end_date, prediction_date, underlying)
P.Predict()

Prices Downloaded
Days To Expiry Downloaded
Simulated Parameters Downloaded
PnL Estimator Initialized
384  documents deleted for 2024-01-18
384  documents entered for 2024-01-18

0  documents deleted for 2024-01-19
384  documents entered for 2024-01-19



In [ ]:
df = pd.DataFrame(client.Strategy.NIFTY_Predicted_Parameters.find())